# Feedback session - 1

Typical solution

In [1]:
import numpy as np


def convolution(M, S):
    M_shape = M.shape
    new_matrix = np.zeros(M_shape)

    for i in range(M_shape[0] - 1):
        for j in range(M_shape[1] - 1):
            for l in [-1, 0, 1]:
                for k in [-1, 0, 1]:
                    new_matrix[i, j] += M[i + l, j + k] * S[l, k]

    return new_matrix

3 Questions:
 - Correctness
 - Style
 - Performance

## Correctness

What happens then arguments are incorrect? Can we inform user about it?

In [2]:
def convolution(M, S):
    if M.ndim != 2 or S.ndim != 2:
        raise ValueError(
            f"Only 2d matrices are supported, {M.ndim}d and {S.ndim}d are provided"
        )
    if S.shape != (3, 3):
        raise ValueError(
            f"Only (3,3) shaped kernels are supported, {S.shape} is provided"
        )

    M_shape = M.shape
    new_matrix = np.zeros(M_shape)

    for i in range(M_shape[0] - 1):
        for j in range(M_shape[1] - 1):
            for l in [-1, 0, 1]:
                for k in [-1, 0, 1]:
                    new_matrix[i, j] += M[i + l, j + k] * S[l, k]

    return new_matrix


convolution(np.zeros((10, 10)), np.zeros((3, 3)))
try:
    convolution(np.zeros((10, 10, 10)), np.zeros((3, 3)))
except ValueError as e:
    print(e)

Only 2d matrices are supported, 3d and 2d are provided


Are there any bugs in output? How could we have noticed this?

In [3]:
convolution(np.identity(10), np.ones((3, 3)))

array([[3., 2., 1., 0., 0., 0., 0., 0., 1., 0.],
       [2., 3., 2., 1., 0., 0., 0., 0., 0., 0.],
       [1., 2., 3., 2., 1., 0., 0., 0., 0., 0.],
       [0., 1., 2., 3., 2., 1., 0., 0., 0., 0.],
       [0., 0., 1., 2., 3., 2., 1., 0., 0., 0.],
       [0., 0., 0., 1., 2., 3., 2., 1., 0., 0.],
       [0., 0., 0., 0., 1., 2., 3., 2., 1., 0.],
       [0., 0., 0., 0., 0., 1., 2., 3., 2., 0.],
       [1., 0., 0., 0., 0., 0., 1., 2., 3., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
def convolution(M, S):
    if M.ndim != 2 or S.ndim != 2:
        raise ValueError(
            f"Only 2d matrices are supported, {M.ndim}d and {S.ndim}d are provided"
        )

    new_matrix = np.zeros((M.shape[0] - S.shape[0], M.shape[1] - S.shape[1]))

    for i in range(new_matrix.shape[0]):
        for j in range(new_matrix.shape[1]):
            for l in range(S.shape[0]):
                for k in range(S.shape[1]):
                    new_matrix[i, j] += M[i + l, j + k] * S[l, k]
    return new_matrix


convolution(np.identity(10), np.ones((2, 2)))

array([[2., 1., 0., 0., 0., 0., 0., 0.],
       [1., 2., 1., 0., 0., 0., 0., 0.],
       [0., 1., 2., 1., 0., 0., 0., 0.],
       [0., 0., 1., 2., 1., 0., 0., 0.],
       [0., 0., 0., 1., 2., 1., 0., 0.],
       [0., 0., 0., 0., 1., 2., 1., 0.],
       [0., 0., 0., 0., 0., 1., 2., 1.],
       [0., 0., 0., 0., 0., 0., 1., 2.]])

## Style

**Enforce PEP-8 compliance, don't do it just by hand!**

Install black, consider pre-commit hooks for future projects. Ctrl+B fixes snippet!

In [5]:
def convolution(M, S):
    np_arr_M = np.asarray(M)
    result = np.zeros(np_arr_M.shape, dtype = int)
    v_ssize, h_ssize = S.shape
    v_msize, h_msize = np_arr_M.shape
    for i in range(v_msize):
      for j in range(h_msize):
        sum = 0
        for l in range(v_ssize):
          for k in range(h_ssize):
            if (0 <= i + l - v_ssize//2 <= v_msize - 1) and (0 <= j + k - h_ssize//2 <= h_msize - 1):
              sum += S[l][k] * np_arr_M[i + l - v_ssize//2][j + k - h_ssize//2]
        result[i][j] = sum
    return result

Also, add description and consider changing variable names to more explicit ones.

Use type annotations. They don't add overhead, they are just hints (aka suggestions).

In [6]:
def convolve(matrix: np.array, kernel: np.array) -> np.array:
    #
    # Convolve 2-dimensional array by 2-dimensional kernel
    # Parameters: 
    #   1) matrix - 2d-nparray, the object of convolution
    #   2) kernel - 2d-nparray, the kernel of convolution, 
    #      kernel MUST have ODD height and wight
    # Returns: 2d-nparray (the same size as matrix's) - the convolved matrix
    #
    
    # kernel center
    center = [x // 2 for x in kernel.shape] 
    # size of original matrix
    size = matrix.shape
    
    result_size = [size[i] + 2 * center[i] for i in range(0,2)]
    result = np.zeros(result_size)
    
    for (x, y), coeff in np.ndenumerate(kernel):
        # coeff = kernel[x,y] - the cofficient of this addition
        result[x : size[0] + x, y : size[1] + y] = np.add( 
            result[x : size[0] + x, y : size[1] + y], 
            matrix * coeff
        ) 
    return result

In [7]:
help(convolve)

Help on function convolve in module __main__:

convolve(matrix: <built-in function array>, kernel: <built-in function array>) -> <built-in function array>



## Performance

How to calculate?

In [8]:
import random, time, timeit


def function():
    while x := random.randint(0, 1000):
        pass


# time.time
start_time = time.time()
function()
total_time = time.time() - start_time
print(f"time.time - {total_time}")

# timeit magic
%timeit function()

# timeit
t = timeit.repeat("function()", "from __main__ import function", repeat=5, number=1000)
print(f"timeit.repeat - {t}")
print(f"\tminimum - {min(t) / 1000}")

time.time - 0.0009992122650146484
671 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
timeit.repeat - [0.6508356000000006, 0.6436627999999995, 0.6351870000000002, 0.6572277, 0.6414837999999996]
	minimum - 0.0006351870000000002


Explain performance jumps.

In [9]:
from scipy import signal
from PIL import Image

image = np.array(Image.open("image.jpg").convert("L"))
kernel = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

%timeit signal.convolve2d(image, kernel)

25.8 ms ± 495 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
def convolution(M, S):
    R = np.zeros((M.shape[0] - S.shape[0] + 1, M.shape[1] - S.shape[1] + 1))
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            for k in range(S.shape[0]):
                for l in range(S.shape[1]):
                    R[i, j] += M[i + k, j + l] * S[k, l]
    return R


%timeit convolution(image, kernel)

6.3 s ± 36.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
def convolution(M, S):
    R = np.zeros((M.shape[0] - S.shape[0] + 1, M.shape[1] - S.shape[1] + 1))
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            R[i, j] = (M[i : i + S.shape[0], j : j + S.shape[1]] * S).sum()
    return R


%timeit convolution(image, kernel)

3.24 s ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def convolution(matrix1, matrix2):
    # Get the dimensions of the input matrices
    rows1, cols1 = matrix1.shape
    rows2, cols2 = matrix2.shape

    # Calculate the output matrix dimensions
    output_rows = rows1 - rows2 + 1
    output_cols = cols1 - cols2 + 1

    # Flip the second matrix horizontally and vertically
    flipped_matrix2 = np.flipud(np.fliplr(matrix2))

    # Pad the input matrix
    padded = np.pad(matrix1, ((rows2 - 1, 0), (cols2 - 1, 0)), mode="constant")

    # Create an array of sliding windows
    windows = np.lib.stride_tricks.sliding_window_view(padded, (rows2, cols2))

    # Element-wise multiply the sliding windows with the flipped matrix
    multiplied = windows * flipped_matrix2

    # Sum along the (rows2, cols2) dimensions
    output = np.sum(multiplied, axis=(2, 3))

    return output


%timeit convolution(image, kernel)

49.6 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
def convolution(matrix, stencil):

    if np.shape(stencil) > np.shape(matrix):
        raise ValueError(
            "Stencil shape is greater than matrix in at least one dimension"
        )

    result = np.zeros(
        np.array(np.shape(matrix)) + np.array(np.shape(stencil)) - np.array((1, 1))
    )

    for biasX in range(np.shape(stencil)[0]):
        for biasY in range(np.shape(stencil)[1]):
            result[
                biasX : np.shape(matrix)[0] + biasX, biasY : np.shape(matrix)[1] + biasY
            ] += (stencil[biasX, biasY] * matrix)

    return result


%timeit convolution(image, kernel)

19.6 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
def convolution(M, S):
    R = np.zeros((M.shape[0] - S.shape[0] + 1, M.shape[1] - S.shape[1] + 1))
    for k in range(S.shape[0]):
        for l in range(S.shape[1]):
            R += M[k : k + R.shape[0], l : l + R.shape[1]] * S[k, l]
    return R


%timeit convolution(image, kernel)

15.6 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
